# Анализ базы данных сервиса для чтения книг

## Цели и ход исследования

Из открытых источников в нашем распоряжении находится база данных крупного сервиса для чтения книг по подписке, содержащая информацию о книгах, издательствах, авторах, а также пользовательские обзоры книг. С целью формулировки ценностного предложения для нового продукта необходимо проанализировать базу данных и сформировать выводы. 

**Цели исследования:**
1. Исследовать таблицы базы данных;
2. Проанализировать базу данных, выполнив SQL-запросы в соответствии с заданиями;
3. Описать результаты выполнения запросов;
4. Сформировать общие выводы.

## Ознакомление с базой данных

In [1]:
# Импортируем необходимые библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

# Установим параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# Сохраним коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

Ознакомимся с базой данных более детально, выведя первые строки каждой из таблиц на экран.

In [2]:
# С помощью SQL-запроса выведем первые 5 строк таблицы "books"
query = '''SELECT *
           FROM books
           LIMIT 5'''

con=engine.connect()

print('Первые строки таблицы "books":')
display(pd.io.sql.read_sql(sql=text(query), con = con))

# Выведем первые 5 строк таблицы "authors"
query2 = '''SELECT *
           FROM authors
           LIMIT 5'''

print('Первые строки таблицы "authors":')
display(pd.io.sql.read_sql(sql=text(query2), con = con))

# Выведем первые 5 строк таблицы "ratings"
query3 = '''SELECT *
           FROM ratings
           LIMIT 5'''

print('Первые строки таблицы "ratings":')
display(pd.io.sql.read_sql(sql=text(query3), con = con))

# Выведем первые 5 строк таблицы "reviews"
query3 = '''SELECT *
           FROM reviews
           LIMIT 5'''

print('Первые строки таблицы "reviews":')
display(pd.io.sql.read_sql(sql=text(query3), con = con))

# Выведем первые 5 строк таблицы "publishers"
query3 = '''SELECT *
           FROM publishers
           LIMIT 5'''

print('Первые строки таблицы "publishers":')
display(pd.io.sql.read_sql(sql=text(query3), con = con))

Первые строки таблицы "books":


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Первые строки таблицы "authors":


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Первые строки таблицы "ratings":


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Первые строки таблицы "reviews":


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Первые строки таблицы "publishers":


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Содержащаяся в таблицах информация соответствует их названию.

## Выполнение SQL-запросов

### Задание 1

Посчитаем, сколько книг вышло после 1 января 2000 года.

In [3]:
# Выполним SQL-запрос, используя Pandas
query = '''SELECT COUNT(book_id) AS counts_of_books
           FROM books
           WHERE publication_date > '2000-01-01'
           '''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,counts_of_books
0,819


Чуть более 800 книг было выпущено с 02 января 2000 года.

### Задание 2

Посчитаем количество обзоров и среднюю оценку для каждой книги.

In [4]:
# Выполним SQL-запрос, используя Pandas
query = '''SELECT b.book_id,
                  b.title AS book_title,
                  COUNT(DISTINCT(rw.review_id)) AS counts_of_reviews,
                  ROUND(AVG(rt.rating), 1) AS average_rating
           FROM books AS b
           LEFT JOIN ratings AS rt ON b.book_id = rt.book_id
           LEFT JOIN reviews AS rw ON b.book_id = rw.book_id
           GROUP BY b.book_id
           '''

con=engine.connect()

book_information = pd.io.sql.read_sql(sql=text(query), con = con)

display(book_information)

print('Максимальное количество обзоров:', book_information['counts_of_reviews'].max())
print('Минимальное количество обзоров:', book_information['counts_of_reviews'].min())
print('----------------------------')
print('Максимальный средний рейтинг:', book_information['average_rating'].max())
print('Минимальный средний рейтинг:', book_information['average_rating'].min())

,book_id,book_title,counts_of_reviews,average_rating
0,1,'Salem's Lot,2,3.7
1,2,1 000 Places to See Before You Die,1,2.5
2,3,13 Little Blue Envelopes (Little Blue Envelope...,3,4.7
3,4,1491: New Revelations of the Americas Before C...,2,4.5
4,5,1776,4,4.0
...,...,...,...,...
995,996,Wyrd Sisters (Discworld #6; Witches #2),3,3.7
996,997,Xenocide (Ender's Saga #3),3,3.4
997,998,Year of Wonders,4,3.2
998,999,You Suck (A Love Story #2),2,4.5


Максимальное количество обзоров: 7
Минимальное количество обзоров: 0
----------------------------
Максимальный средний рейтинг: 5.0
Минимальный средний рейтинг: 1.5


Количество обзоров на книги варьируется от 0 до 7. 

Рейтинг имеет широкий диапазон значений, изменяющийся от 1.5 до 5.

### Задание 3

Определим издательство, выпустившее наибольшее число книг толще 50 страниц.

In [5]:
# Выполним SQL-запрос, используя Pandas
query = '''WITH
           books_more_50 AS (SELECT *
                             FROM books
                             WHERE num_pages > 50)
                         
           SELECT p.publisher,
                  COUNT(b.book_id) AS counts_of_books
           FROM books_more_50 AS b
           JOIN publishers AS p ON b.publisher_id = p.publisher_id
           GROUP BY p.publisher
           ORDER BY counts_of_books DESC
           LIMIT 1
           '''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,counts_of_books
0,Penguin Books,42


Издательство "Penguin Books" выпустило наибольшее число книг толще 50 страниц – 42 единицы.

### Задание 4

Определим автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками.

In [6]:
# Выполним SQL-запрос, используя Pandas
query = '''WITH
           books_with_rating_more_50 AS (SELECT b.book_id,
                                                b.author_id,
                                                b.title AS book_title,
                                                AVG(rt.rating) AS rating,
                                                COUNT(rt.rating_id) AS counts_of_ratings
                                         FROM books AS b
                                         LEFT JOIN ratings AS rt ON b.book_id = rt.book_id
                                         GROUP BY b.book_id
                                         HAVING COUNT(rt.rating_id) >= 50)
                                         
           SELECT a.author,
                  ROUND(AVG(b.rating), 2) AS average_rating
           FROM books_with_rating_more_50 AS b
           LEFT JOIN authors AS a ON b.author_id = a.author_id
           GROUP BY a.author
           ORDER BY average_rating DESC
           LIMIT 1
           '''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,author,average_rating
0,J.K. Rowling/Mary GrandPré,4.28


J.K. Rowling совместно с Mary GrandPré являются авторами с самыми рейтинговыми книгами. Все без ума Гарри Поттера!

### Задание 5

Посчитаем среднее количество обзоров от пользователей, которые поставили более 48 оценок.

In [7]:
# Выполним SQL-запрос, используя Pandas
query = '''WITH
           users_with_reviews_more_48 AS (SELECT rt.username,
                                                 COUNT(rt.rating_id) AS counts_of_ratings
                                          FROM books AS b
                                          LEFT JOIN ratings AS rt ON b.book_id = rt.book_id
                                          GROUP BY rt.username
                                          HAVING COUNT(rt.rating_id) > 48),
                                          
           counts_of_reviews AS (SELECT rw.username,
                                        COUNT(rw.review_id) AS counts_of_reviews
                                 FROM reviews AS rw
                                 RIGHT JOIN users_with_reviews_more_48 AS u ON u.username = rw.username
                                 GROUP BY rw.username)
                                 
           SELECT AVG(counts_of_reviews) AS average_counts_of_reviews
           FROM counts_of_reviews
           '''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,average_counts_of_reviews
0,24.0


В среднем, наиболее активные пользователи, поставившие более 48 оценок, пишут около 24 обзоров.

## Выводы

По итогам проведенного исследования были сделаны следующие выводы:

- С 02 января 2000 года было выпущено чуть более 800 книг;
- Количество обзоров на книги варьируется от 0 до 7;
- Рейтинг имеет широкий диапазон значений, изменяющийся от 1.5 до 5;
- Издательство "Penguin Books" выпустило наибольшее число книг толще 50 страниц – 42 единицы;
- J.K. Rowling совместно с Mary GrandPré являются авторами с самыми рейтинговыми книгами;
- В среднем, наиболее активные пользователи, поставившие более 48 оценок, пишут около 24 обзоров.